In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import datetime
import math
from google.colab import drive


In [2]:
#Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd gdrive/MyDrive/DatabaseAll/Armidale/

/content/gdrive/MyDrive/DatabaseAll/Armidale


In [4]:
ls

2017_Armidale.csv            2020_Armidale.csv  model-autumn/
2017_Armidale_processed.csv  Armidale.csv       path/
2018_Armidale.csv            Armidale.gsheet    tune_parameters_LSTM/
2018_Armidale_processed.csv  case2.csv
2019_Armidale.csv            datapredict.npy


In [5]:
#Reading Database
df = pd.read_csv(r'2019_Armidale.csv', header = 0)
df.head()

,Period,kWh Value,kVARh Value,kW,kVAR,Max kW,Demand kVA,PF,CO2,No Of Meters,kWh Actual,kWh Estimate,Minimum Temperature,TOU Demand kW,TOU Demand kVA
0,01/01/2019 00:00:00,17.567,2.912,70.268,11.648,70.268,71.227,0.9865,14.5806,1.0,17.567,0,20.5,0.0,0.0
1,01/01/2019 00:15:00,17.504,3.264,70.016,13.056,70.016,71.223,0.9831,14.5283,1.0,17.504,0,20.5,0.0,0.0
2,01/01/2019 00:30:00,18.880,4.095,75.520,16.380,75.520,77.276,0.9773,15.6704,1.0,18.880,0,20.0,0.0,0.0
3,01/01/2019 00:45:00,18.272,4.192,73.088,16.768,73.088,74.987,0.9747,15.1658,1.0,18.272,0,20.0,0.0,0.0
4,01/01/2019 01:00:00,17.888,2.495,71.552,9.980,71.552,72.245,0.9904,14.8470,1.0,17.888,0,20.1,0.0,0.0


In [6]:
#Armidale School Information for 2020
holiday = [[datetime.datetime.strptime('2019-01-01', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-01-01', '%Y-%m-%d').date()], 
           [datetime.datetime.strptime('2019-01-28', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-01-28', '%Y-%m-%d').date()], 
           [datetime.datetime.strptime('2019-04-19', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-04-22', '%Y-%m-%d').date()], 
           [datetime.datetime.strptime('2019-04-25', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-04-25', '%Y-%m-%d').date()], 
           [datetime.datetime.strptime('2019-06-07', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-06-10', '%Y-%m-%d').date()], 
           [datetime.datetime.strptime('2019-10-07', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-10-07', '%Y-%m-%d').date()],
           [datetime.datetime.strptime('2019-12-25', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-12-26', '%Y-%m-%d').date()]
          ]
school_term = [[datetime.datetime.strptime('2019-01-24', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-04-12', '%Y-%m-%d').date()],
               [datetime.datetime.strptime('2019-04-29', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-06-28', '%Y-%m-%d').date()],
               [datetime.datetime.strptime('2019-07-22', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-09-27', '%Y-%m-%d').date()], 
               [datetime.datetime.strptime('2019-10-14', '%Y-%m-%d').date(), datetime.datetime.strptime('2019-12-05', '%Y-%m-%d').date()],
               ]
working_time = [[datetime.datetime.strptime('08:30:00', '%H:%M:%S').time(), datetime.datetime.strptime('17:00:00', '%H:%M:%S').time()]]



In [7]:
#Some pre-processing Functions
def getSeason(date):
    """
        Determine the natural season
    """
    month = date.month
    if(month == 12 or month == 1 or month == 2):
        return 'Summer' #summer
    elif(month == 3 or month == 4 or month == 5):
        return 'Autumn' #autumn
    elif(month == 6 or month == 7 or month == 8):
        return 'Winter' #winter
    else: 
        return 'Spring' #spring

def getIsWeekday(date):
    """
        Check if a date is weekend or not
    """
    return 1 if (date.weekday() < 5) else 0
def getIsSchoolTerm(date, query_result = school_term):
    """
        Check if a date is school term
    """
    for p in query_result:
        if (date >= p[0] and date <= p[1]):
            return 1
    return 0    

def getIsHoliday(date, query_result = holiday):
    """
        Check if a date is holiday
    """
    for p in query_result:
        if (date >= p[0] and date <= p[1]):
            return 0
    return 1
def getIsWorkingTime(time, query_result = working_time):
    """
        Check if a time is working time
    """
    
    for p in query_result:
        if (time >= p[0] and time <= p[1]):
            return 1
    return 0

In [8]:
df.head()

,Period,kWh Value,kVARh Value,kW,kVAR,Max kW,Demand kVA,PF,CO2,No Of Meters,kWh Actual,kWh Estimate,Minimum Temperature,TOU Demand kW,TOU Demand kVA
0,01/01/2019 00:00:00,17.567,2.912,70.268,11.648,70.268,71.227,0.9865,14.5806,1.0,17.567,0,20.5,0.0,0.0
1,01/01/2019 00:15:00,17.504,3.264,70.016,13.056,70.016,71.223,0.9831,14.5283,1.0,17.504,0,20.5,0.0,0.0
2,01/01/2019 00:30:00,18.880,4.095,75.520,16.380,75.520,77.276,0.9773,15.6704,1.0,18.880,0,20.0,0.0,0.0
3,01/01/2019 00:45:00,18.272,4.192,73.088,16.768,73.088,74.987,0.9747,15.1658,1.0,18.272,0,20.0,0.0,0.0
4,01/01/2019 01:00:00,17.888,2.495,71.552,9.980,71.552,72.245,0.9904,14.8470,1.0,17.888,0,20.1,0.0,0.0


In [9]:
df[['Period']].head()

,Period
0,01/01/2019 00:00:00
1,01/01/2019 00:15:00
2,01/01/2019 00:30:00
3,01/01/2019 00:45:00
4,01/01/2019 01:00:00


In [10]:
#Transforming the Data
df['Date'] = df['Period'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S').date())
df['Time'] = df['Period'].apply(lambda x: datetime.datetime.strptime(x, '%m/%d/%Y %H:%M:%S').time())
df['Demand']=df['kW']
df['Temp']=df['Minimum Temperature']
df['Season']=df['Date'].apply(lambda x: getSeason(x))
df = df.drop(columns = ['Period', 'kWh Value', 'kVARh Value', 'kVAR', 'Max kW','kW','Demand kVA', 'PF', 'CO2', 'No Of Meters', 'kWh Actual',
                        'kWh Estimate', 'Minimum Temperature','TOU Demand kW','TOU Demand kVA', ])

In [11]:
df.columns

Index(['Date', 'Time', 'Demand', 'Temp', 'Season'], dtype='object')

In [12]:
#Transforming the Data
df['isWeekend'] = df['Date'].apply(lambda x: getIsWeekday(x)) # 0 if is weekend
df['isTermBreak'] = df['Date'].apply(lambda x: getIsSchoolTerm(x)) # 0 if is school term
df['isHoliday'] = df['Date'].apply(lambda x: getIsHoliday(x)) # 0 if is holiday
df['isWorkingTime'] = df['Time'].apply(lambda x: getIsWorkingTime(x)) # 1 if is working time
df['occupancy'] = df.apply(lambda row: (row.isWeekend and row.isTermBreak and row.isHoliday) and row.isWorkingTime, axis=1)

In [13]:
df.columns

Index(['Date', 'Time', 'Demand', 'Temp', 'Season', 'isWeekend', 'isTermBreak',
       'isHoliday', 'isWorkingTime', 'occupancy'],
      dtype='object')

In [14]:
df.describe()

,Demand,Temp,isWeekend,isTermBreak,isHoliday,isWorkingTime,occupancy
count,35040.000000,35036.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,121.445431,14.385090,0.715068,0.715068,0.961644,0.364583,0.187785
std,55.317873,7.353486,0.451388,0.451388,0.192057,0.481320,0.390546
min,0.000000,-5.500000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,76.032000,8.700000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,113.792000,14.200000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,160.000000,19.300000,1.000000,1.000000,1.000000,1.000000,0.000000
max,312.320000,35.700000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           35040 non-null  object 
 1   Time           35040 non-null  object 
 2   Demand         35040 non-null  float64
 3   Temp           35036 non-null  float64
 4   Season         35040 non-null  object 
 5   isWeekend      35040 non-null  int64  
 6   isTermBreak    35040 non-null  int64  
 7   isHoliday      35040 non-null  int64  
 8   isWorkingTime  35040 non-null  int64  
 9   occupancy      35040 non-null  int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 2.7+ MB


In [16]:
#Check if Demand is 0 and consequently any NaN values in Temperature
df[df['Demand']==0]

,Date,Time,Demand,Temp,Season,isWeekend,isTermBreak,isHoliday,isWorkingTime,occupancy
26696,2019-10-06,02:00:00,0.0,NaN,Spring,0,0,1,0,0
26697,2019-10-06,02:15:00,0.0,NaN,Spring,0,0,1,0,0
26698,2019-10-06,02:30:00,0.0,NaN,Spring,0,0,1,0,0
26699,2019-10-06,02:45:00,0.0,NaN,Spring,0,0,1,0,0


In [17]:
#Replace the Demand 0 values to NaN and replace temperature NaN values by previous values
df[['Demand']] = df[['Demand']].replace(0.0,np.nan)
df[['Demand','Temp']] = df[['Demand','Temp']].fillna(method='ffill')

In [18]:
df.loc[26695:26700]

,Date,Time,Demand,Temp,Season,isWeekend,isTermBreak,isHoliday,isWorkingTime,occupancy
26695,2019-10-06,01:45:00,60.544,11.3,Spring,0,0,1,0,0
26696,2019-10-06,02:00:00,60.544,11.3,Spring,0,0,1,0,0
26697,2019-10-06,02:15:00,60.544,11.3,Spring,0,0,1,0,0
26698,2019-10-06,02:30:00,60.544,11.3,Spring,0,0,1,0,0
26699,2019-10-06,02:45:00,60.544,11.3,Spring,0,0,1,0,0
26700,2019-10-06,03:00:00,67.072,10.5,Spring,0,0,1,0,0


In [19]:
df.columns

Index(['Date', 'Time', 'Demand', 'Temp', 'Season', 'isWeekend', 'isTermBreak',
       'isHoliday', 'isWorkingTime', 'occupancy'],
      dtype='object')

In [20]:
df.to_csv('2019_Armidale_processed.csv')